# Lab 01. Градиенты в Pytorch

## Autograd - Автоматическое дифференцирование

В первой лабораторной работе, мы рассмотрим концепцию динамического вычислительного графа, который состоит из всех объектов Tensor, связанных в сеть, а также функций, используемых для их создания. Обратите внимание, что только те входные тензоры, которые мы создаем сами, не будут иметь связанных объектов Function.

Пакет autograd из PyTorch обеспечивает автоматическое дифференциорование для всех операций на Tensors. Операции являются атрибутами самих тензоров.

Когда для атрибута .requires_grad у Tensor  установлено значение True, он начинает отслеживать все операции над ним. Когда операция заканчивается, вы можете вызвать .backward () и автоматически рассчитать все градиенты. Градиент для тензора будет накапливаться в его атрибуте .grad.

## Backpropagation за один шаг
Мы начнем с применения одной полиномиальной функции $y = f(x)$ к тензору $x$. Затем выполним backprop для подсчета $\frac {dy} {dx}$.

$$\quad y= 2x^4 + x^3 + 3x^2 + 5x + 1$$

$$\quad y'= 8x^3 + 3x^2 + 6x + 5$$

#### Step 1. Импорт

In [14]:
import torch

#### Step 2. Создаем тензор. <tt>requires_grad</tt> устанавливаем в  True
Это устанавливает отслеживание вычислений на тензоре.

In [15]:
x = torch.tensor(2.0, requires_grad=True)

In [16]:
x

tensor(2., requires_grad=True)

#### Step 3. Задаем функцию

In [17]:
y = 2*x**4 + x**3 + 3*x**2 + 5*x + 1

print(y)

tensor(63., grad_fn=<AddBackward0>)


Поскольку $y$ была создана в результате операции, у нее есть связанная функция градиента, доступная через <tt> y.grad_fn </tt> <br>
Вычисление $y$ для данного $x=2$ выполняется следующим образом: <br>

$\quad y=2(2)^4+(2)^3+3(2)^2+5(2)+1 = 32+8+12+10+1 = 63$

Значение $y=63$ при $x=2$.

#### Step 4. Backprop

In [18]:
y.backward()

#### Step 5. Показать полученный градиент

In [19]:
print(x.grad)

tensor(93.)


Обратите внимание, что <tt> x.grad </tt> является атрибутом тензора $x$, поэтому мы не используем скобки. Вычисление является результатом <br>

$ \quad y'= 8 (2)^3 + 3 (2)^2 + 6 (2) +5 = 64 + 12 + 12 + 5 = 93 $

Это значение производной в точке $ (2,63) $.

## Back-propagation с несколькими шагами
Теперь давайте сделаем что-то более сложное, включив слои $ y $ и $ z $ между $ x $ и нашим выходным слоем $ out $.
#### 1. Создаем тензор

In [20]:
x = torch.tensor([[1.0, 2.0, 3.0],[3.0, 2.0, 1.0]], requires_grad=True)
print(x)

tensor([[1., 2., 3.],
        [3., 2., 1.]], requires_grad=True)


#### 2. Создаем первый слой $y = 3x+2$

In [21]:
y = 3*x + 2
print(y)

tensor([[ 5.,  8., 11.],
        [11.,  8.,  5.]], grad_fn=<AddBackward0>)


#### 3. Создаем второй слой $z = 2y^2$

In [22]:
z = 2*y**2
print(z)

tensor([[ 50., 128., 242.],
        [242., 128.,  50.]], grad_fn=<MulBackward0>)


#### 4. Выходное значение $out$ считаем как среднее

In [23]:
out = z.mean()
print(out)

tensor(140., grad_fn=<MeanBackward0>)


#### 5. Выполняем back-propagation чтобы найти градиент от $out$ по x


In [24]:
out.backward()
print(x.grad)

tensor([[10., 16., 22.],
        [22., 16., 10.]])


Вы должны увидеть матрицу 2х3. Если мы обозначим финальный тензор <tt> out </tt> через «$ o $», мы можем вычислить частную производную $ o $ по $ x_i $ следующим образом: <br>

$o = \frac {1} {6}\sum_{i=1}^{6} z_i$<br>

$z_i = 2(y_i)^2 = 2(3x_i+2)^2$<br>

Для решения производной от $ z_i $ мы используем chain rule (дифференцирование сложной функции), то есть производная от $f(g(x)) = f'(g(x))g'(x)$<br>

В этом случае<br>

$\begin{split} f(g(x)) &= 2(g(x))^2, \quad &f'(g(x)) = 4g(x) \\
g(x) &= 3x+2, &g'(x) = 3 \\
\frac {dz} {dx} &= 4g(x)\times 3 &= 12(3x+2) \end{split}$

Поэтому,<br>

$\frac{\partial o}{\partial x_i} = \frac{1}{6}\times 12(3x+2)$<br>

$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = 2(3(1)+2) = 10$

$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=2} = 2(3(2)+2) = 16$

$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=3} = 2(3(3)+2) = 22$

### Отключить отслеживание градиента
Могут быть случаи, когда не нужно отслеживать историю вычислений.

Вы можете выставить  атрибут <tt> require_grad </tt>, используя `.requires_grad_ (True)` (или False) по мере необходимости.

При выполнении вычислений часто полезно обернуть набор операций в блок `with torch.no_grad(): {...}`


Другой метод - запуск `.detach()` для тензора, чтобы предотвратить отслеживание будущих вычислений. Это может быть удобно при клонировании тензора.

In [25]:
x.requires_grad = False
y = 3*x + 2
# backward вызовет ошибку, тк подсчет градиентов отключен
#y.backward()
print(y)

tensor([[ 5.,  8., 11.],
        [11.,  8.,  5.]])
